# CGRtools Tutorial

(c) 2019, Ramil Nugmanov

install CGRtools package forst: `pip install CGRtools`

NOTE: tutorial must be performed sequentially. Random cell running will lead to unexpected results.

In [ ]:
# load data for tutorial
from pickle import load
from traceback import format_exc

with open('molecules.dat', 'rb') as f:
    molecules = load(f) # list of MoleculeContainer objects
with open('reactions.dat', 'rb') as f:
    reactions = load(f) # list of ReactionContainer objects

m1, m2, m3, m4 = molecules # molecule
m7 = m3.copy()
m11 = m3.copy()
m11.standardize()
m12 = m3.copy()
m7.standardize()
r1 = reactions[0] # reaction
m5, m6 = r1.reagents[:2]
m8 = m7.substructure([4, 5, 6, 7, 8, 9], as_view=False)
m9 = m6.substructure([5, 6,7, 8], as_view=False) # acid
m10 =  r1.products[0].copy()

# 1. Data types and operations with them

CGRtools has subpackage containers with data structures classes:
* MoleculeContainer - molecule structure
* ReactionContainer - reaction structure
* CGRContainer - CGR structure
* QueryContainer - for substructure searching in molecules
* QueryCGRContainer - for substructure searching in CGRs

In [ ]:
from CGRtools.containers import * # import all containers

## 1.1. MoleculeContainer
Molecules is undirected graphs. Molecules contains atom objects and bond objects.

Atoms stored in dictionary keyed with unique number of each atoms.  
Bonds stored in sparse matrix keyed by atoms numbers between which bonds exists.

Molecules has next methods and properties:

In [ ]:
m1.meta # dictionary for molecule metadata storing. for example used for storing MDL SDF DTYPE/DATUM fields

In [ ]:
m1 # molecules support depiction and graphic representation in jupyter notebooks.

In [ ]:
m1.depict() # depiction return SVG image format string

In [ ]:
with open('molecule.svg', 'w') as f: # save image to file
    f.write(m1.depict())

In [ ]:
m_copy = m1.copy() # isolated copy of molecule
m_copy

In [ ]:
len(m1) # get number of atoms in molecule
# or 
m1.atoms_count

In [ ]:
m1.bonds_count # number of bonds

In [ ]:
m1.atoms_numbers # list of available atoms numbers

In [ ]:
# this method calculate additional atoms attributes: number of connected neighbors and hybridization. see below for usage
m1.reset_query_marks() # by default this attributes is None (speed-up)
m3.reset_query_marks()

Hybridizations of atoms:
* 1 (s) - all bonds of atom is single
* 2 (d) - atom has one double bond and other is single
* 3 (t) - atom has one triple or two double bonds and other is single
* 4 (a) - atom in aromatic ring

Neighbors and hybridizations atom attributes needs for **substructure operations** and structure standardization. See below

In [ ]:
# iterate over atoms with its numbers
list(m1.atoms())  # works same as dict.items()

In [ ]:
# iterate over bonds with connected atoms numbers
list(m1.bonds())

In [ ]:
# access to atom by number
m1.atom(1)

In [ ]:
try:
    m1.atom(10) # raise error for nonexistent atom numbers
except KeyError:
    print(format_exc())

In [ ]:
# access to bond by atoms numbers
m1.bond(1, 4)

In [ ]:
try:
    m1.bond(1, 3) # raise error for nonexistent bond
except KeyError:
    print(format_exc())

#### Atom objects is dict-like classes which store information about:
* element
* isotope
* charge
* multiplicity
* xyz coordinates

Also atoms has methods for data integrity checks and some internal data.

In [ ]:
a = m1.atom(1)

# access to information
a.element # element symbol
# or
a['element']

In [ ]:
a.charge # formal charge
# or
a['charge']

In [ ]:
a.multiplicity # atom multiplicity. None if not set
# or
a['multiplicity']

In [ ]:
a.isotope # atom isotope. default if not set. defaults isotopes same as in INCHI trust
# or
a['isotope']

In [ ]:
a.x # coordinates
a.y
a.z
# or
a['x']
a['y']
a['z']

In [ ]:
a.neighbors # this attribute calculated and read-only. see reset_query_marks above

In [ ]:
a.hybridization # this attribute calculated and read-only

In [ ]:
try:
    a.hybridization = 2 # read-only!
except AttributeError:
    print(format_exc())

#### Presented attributes is editable.

CGRtools checks data before changing

In [ ]:
a.isotope = 16
# or 
a['isotope'] = 16

In [ ]:
m1.flush_cache() # due to caching (speed-up) for observing changes in atoms and bonds need to reset cache of molecule
m1

In [ ]:
try:
    a.isotope = 0 # raise error
except ValueError:
    print(format_exc())

In [ ]:
# bond objects also is dict-like classes which store information about order
b = m1.bond(3, 4)

b.order
#
b['order']

In [ ]:
b.order = 1 # order change also possible
# or
b['order'] = 1

In [ ]:
m1.flush_cache()
m1

In [ ]:
try:
    b['order'] = 0 # raises error
except ValueError:
    print(format_exc())

For bond removing You should to use `delete_bond` method 

In [ ]:
m1.delete_bond(3, 4)
m1

For atom removing `delete_atom` method exists

In [ ]:
m1.delete_atom(3)
m1

In [ ]:
m_copy # copy unchanged!

Atoms and bonds objects can be converted into integer representation.

Each bit-set in this incheger code information about attributes.  
Atoms has 21 bit code (rounded to 32 bit int):
* 7b (highest bits) atom number (2 ** 7 - 1 == 127, currently 118 elements presented in table)
* 9b isotope (511, highest known isotope ~300)
* 3b charge. charges from range -3 - +3 rescaled to range 0-6
* 2b multiplicity.

In [ ]:
int(a)
# 131596 == 0001000 000010000 011 00
# 0001000 == 8 Oxygen
# 000010000 == 16 isotope
# 011 == 3 (3 - 3 = 0) uncharged
# 00 == 0 hasn't multiplicity

In [ ]:
int(b)  # bonds coded by it's order

In [ ]:
print(m1.atom_implicit_h(1)) # get number of implicit hydrogens on atom 1
print(m1.atom_explicit_h(1)) # get number of explicit hydrogens on atom 1
print(m1.atom_total_h(1)) # get total number of hydrogens on atom 1

In [ ]:
m1

In [ ]:
m1.check_valence() # return list of numbers of atoms with invalid valences

In [ ]:
m4 # molecule with errors

In [ ]:
m4.check_valence()

In [ ]:
m3

In [ ]:
m3.sssr # list or lists of atoms forming rings

#### Connected components
Sometimes molecules has disconnected components (salts etc).

You can see it and split to separate molecules.

In [ ]:
m2

In [ ]:
m2.connected_components # list of list of atoms of components

In [ ]:
anion, cation = m2.split() # split molecule to components

In [ ]:
anion

In [ ]:
cation

#### Union of molecules
Sometimes salts presented in reaction as anions set. This presentation is ambiguous.

For example: **Ag+** + **NO3-** + **Na+** + **Br-** = **Ag+** + **Br-** + **Na+** + **NO3-**. Reagents and products set is equal

You can combine pairs of anion-cation to single molecule

In [ ]:
salt = anion | cation # or anion.union(cation)
salt # this salt has disconnected components, but now this is single compound

#### From molecules possible to extract substructures.

By default returned objects is read-only projection of original molecule (except attributes of atoms/bonds).  
Changes in original molecule (bond breaking/formation, atom insertion/deletion, atom/bond attributes changes) will be mirrored in projection.  
Projections share original molecule neighbors and hybridization attributes

In [ ]:
proj = m3.substructure([4,5,6,7,8,9])  # projection to molecule part.
proj

In [ ]:
m3.atom(4).neighbors

In [ ]:
proj.atom(4).neighbors # same as in original molecule

In [ ]:
from networkx.exception import NetworkXError
try:
    proj.reset_query_marks() # projections has blocked methods for molecule changing
except NetworkXError:
    print(format_exc())

In [ ]:
benzene = m3.substructure([4,5,6,7,8,9], as_view=False) # isolated copy (not projection)
benzene

In [ ]:
benzene.atom(4).neighbors is None # empty attribute. benzene is new molecule. need to call reset_query_marks

In [ ]:
benzene.reset_query_marks()

In [ ]:
benzene.atom(4).neighbors # now neighbors number of atom 4 is 2.

* Projection of projection also projection of original molecule
* Projection can be converted to isolated molecule by calling method copy()

In [ ]:
proj_copy = proj.copy()
proj_copy

In [ ]:
proj_copy.reset_query_marks() # this not projection. this new molecule!

Changes mirroring example

In [ ]:
m3.delete_bond(4, 5)
m3

In [ ]:
proj.flush_cache() # remove cached image
proj

`augmented_substructure` is substructure extended with neighbors atoms.
**deep** argument give control to number of included shells of substructure core.

In [ ]:
aug = m3.augmented_substructure([10], deep=2) # also projection by default. atom 10 is Nitrogen
aug

In [ ]:
aug.atom(10).hybridization

#### Atoms Ordering
`atoms_order` property return dictionary of atom numbers with it's prime-value orders. Equal atoms has equal values. Morgan algorithm used.


In [ ]:
m5.atoms_order

#### Atoms number can be changed by `remap` method.

First argument is dict keyed with existing numbers and valued with new. Possible to set only part of atoms. Argument *copy* by default is False. If copy True - new object will be created, else existing molecule will be changed.

In [ ]:
for n, a in m5.atoms():
    print(n, a.element)
for n, m, b in m5.bonds():
    print(m, n, b.order)

In [ ]:
remapped = m5.remap({4:2}, copy=True)
remapped

In [ ]:
for n, a in remapped.atoms():
    print(n, a.element)
for n, m, b in remapped.bonds():
    print(m, n, b.order)

## 1.2. ReactionContainer

ReactionContainer objects has next properties:
* **reagents** - list of reagents molecules
* **reactants** - list of reactants molecules
* **products** - list of products molecules
* **meta** - dictinary of reaction metadata (DTYPE/DATUM block in RDF)

In [ ]:
r1 # depiction supported

In [ ]:
r1.meta

In [ ]:
print(r1.reagents, r1.products)  # access to lists of reagents and products. printed molecules signatures. see below
reagent1, reagent2, reagent3 = r1.reagents
product = r1.products[0]

Reactions also has `standardize`, `aromatize`, `reset_query_marks`, `implicify_hydrogens` and `explicify_hydrogens` methods (see part 3). This methods are applied independently to every molecule in reaction.

## 1.3. CGR
CGRConatainer object is similar to molecules, except some methods. The following methods are not suppoted:
* aromatize
* standardize
* implicify_hydrogens
* explicify_hydrogens
* atom_implicit_h
* atom_explicit_h
* atom_total_h
* check_valence

CGR specific methods:
* centers_list
* center_atoms
* center_bonds

CGR container is undirected graph. atoms and bonds in CGR has two states: reagent and product

#### Composing to CGR

As mentioned above, molecules internally store atoms keyed by unique numbers.  
This numbers used for atom-to-atom mapping in CGRtools.
This gives to us easy approach for constructing of CGRs.

Pairs of molecules can be transformed to CGR. Same atom numbers imply the same atoms.

Reaction also can be composed into CGR. Atom numbers of molecules used as Atom-To-Atom mapping of reagents to products.

In [ ]:
cgr1 = m7 ^ m8 # or m7.compose(m8)
print(cgr1)  # CGR object currently don't support depiction. signature printed. see below

This is CGR. U can see changed bonds connected to ring
![cgr1.png](attachment:cgr1.png)

In [ ]:
r1

In [ ]:
cgr2 = ~r1 # or  r1.compose()
print(cgr2) # signature printed. see below

![cgr2.png](attachment:cgr2.png)

In [ ]:
cgr2.reset_query_marks() # CGRs also has reset_query_marks

In [ ]:
a = cgr2.atom(2) # atom access same as in molecule

In [ ]:
a.element # element attribute
# or a['element']

In [ ]:
a.isotope # isotope attribute
# or a['isotope']

In CGR attributes charge, multiplicity, x y z, neighbors and hybridization refer to atom state in reagent part in reaction equation;  arguments p_charge, p_multiplicity, p_x p_y p_z, p_neighbors and p_hybridization could be used to extract atom state in product part in equation.

In [ ]:
a.charge # reagent state charge
# or 
a['charge']

In [ ]:
a.p_charge # product state charge
#or 
a['p_charge']

In [ ]:
a.p_multiplicity

In [ ]:
a.p_x
a.p_y
a.p_z

In [ ]:
a.neighbors

In [ ]:
a.p_neighbors

In [ ]:
a.hybridization

In [ ]:
a.p_hybridization

In [ ]:
b = cgr1.bond(4, 10)

#### Bonds has order and p_order attribute
If order attribute value is None, this indicate bond formation  
If p_order is None, this indicate bond breaking

Both order and p_order can't be None

In [ ]:
b.order

In [ ]:
b.p_order is None

CGR can be decomposed back to molecules of reagent and product state.

However CGR is lose information about unbalanced reactions. this may lead to strange structures

In [ ]:
reagent_part, product_part = ~cgr1 # or cgr1.decompose() 

In [ ]:
reagent_part # united reagents

In [ ]:
product_part # united products. Original product is benzene molecule only.

For decomposing CGR into reaction object You can use `CGRpreparer` class

In [ ]:
from CGRtools import CGRpreparer
preparer = CGRpreparer()

In [ ]:
decomposed = preparer.decompose(cgr2)

In [ ]:
decomposed # You can see balansed water in products. 
# This is side-effect of CGR decomposing. but works correctly only for atom-unbalansed reactions with balansed elenctrons
# In next release electron balansing will be added.

In [ ]:
r1 # original reaction

## 1.4 Queries

CGRtools support Queries. Queries use in isomorphism additional information about atoms: neighbors and hybridization

Queries don't has `reset_query_marks` method

In [ ]:
from CGRtools.containers import*

In [ ]:
m10.reset_query_marks()
m10 # ether

In [ ]:
carb = m10.substructure([5,7,8, 2]) # carboxyl fragment (projection)
carb

In [ ]:
q = QueryContainer(carb)  # convert molecule into query
print(q) # QueryContainer don't support depiction yet

CGRs also can be transformed into Query.

`QueryCGRContainer` is similar to QueryContainer class for CGRs and has same API.

QueryCGRContainer take into account reagent and product side state of atoms and bonds including neighbors and hybridization

In [ ]:
cgr1.reset_query_marks()
cgr_q = QueryCGRContainer(cgr1)
print(cgr_q)

## 1.5. Molecules, CGRs, Reactions construction

CGRtools has API for construction of objects.

CGR and Molecule has methods `add_atom` and `add_bond` for adding atoms and bonds.

In [ ]:
from CGRtools.containers import *

In [ ]:
m = MoleculeContainer() # new empty molecule

m.add_atom('C')  # add Carbon atom
m.add_atom(6) # add Carbon atom. {'element': 6} - not valid!
m.add_atom({'element': 'O', 'charge': -1}) # add charged Oxygen atom

# add_atom has second argument for setting atom number. by default next after biggest will be set.
m.add_atom({'element': 'Na', 'charge': 1}, 4)

{'element': 'O', 'charge': -1} keys in dictionary is atom attributes. see above

In [ ]:
m.add_bond(1, 2, 1) # add bond with order = 1 between atoms 1 and 2
m.add_bond(3, 2, {'order': 1})

In [ ]:
m.calculate2d()
m

Reactions can be constructed from molecules

In [ ]:
r = ReactionContainer() # empty reaction
r.reagents.append(m1)
r.products.append(m3)
# or
r = ReactionContainer(reagents=[m1], products=[m11]) # filled reaction
# or
r = ReactionContainer([m1], [m11]) # equal to second

In [ ]:
r

In [ ]:
r.products.append(m.copy()) # reagents, products, reactants attributes is list-like. You can add/remove molecules directly to attributes
r.flush_cache()

In [ ]:
r # molecules coordinates automatically not changed.

In [ ]:
r.fix_positions() # this method fix molecules coordinates.
r

QueryContainers can be constructed same as molecules.

QueryContainers additionally support atoms, neighbors and hybridization lists

In [ ]:
q = QueryContainer()
q.add_atom({'element': ['N', 'O']}) # N or O atom, any isotope, neutral charged, no difference in neighbors count and hybridization
q.add_atom({'element': 'C', 'neighbors': [2, 3]}) # Carbon, neutral charged, 2 or 3 non-hydrogen heignbors
q.add_atom({'element': 'O'})
q.add_bond(1, 2, 1)
q.add_bond(2, 3, 2)
# any Amide or Carboxilic group

In [ ]:
print(q)

## 1.6. Extending CGRtools

You can easily customize CGRtools for your tasks.  
CGRtools is OOP-oriented library with subclassing support.

For example You can add special marks on atoms for ligand donor centers.

In [ ]:
from CGRtools.containers import MoleculeContainer
from CGRtools.attributes import Atom

In [ ]:
class MarkedAtom(Atom):
    __slots__ = '__mark' # all new attributes should be slotted!
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.__mark = None # set default value

    @property
    def mark(self):
        return self.__mark
    
    @mark.setter
    def mark(self, mark):
        # do some checks and calculations
        self.__mark = mark


In [ ]:
class MarkedMoleculeContainer(MoleculeContainer):
    node_attr_dict_factory = MarkedAtom # override atom container

In [ ]:
m = MarkedMoleculeContainer()
m.add_atom('C')
m.add_atom('N')
m.add_bond(1, 2, 1)

m.atom(2)['mark'] = 1 # in this example dicts setitem supported but update not.

In [ ]:
m.atom(2).mark

# 2. Signatures

## 2.1. Molecule Signatures
MoleculeContainer has methods for unique molecule signature generation
signature is SMILES string with explicit bonds notation and canonical atoms ordering. For pyroles SMILES does not comply with the rules.

For signature generation need to call `str` function on molecule object.  
Also possible to generate hashed signature (SHA512) by calling `bytes` function on molecule.

Order of atoms calculated by Morgan-like algorithm.
On initial state for each atoms it's integer code calculated. 
All bonds from atoms also coded as integers and stored in sorted tuple.
Atom code and tuple of it's bonds used for ordering and similar atoms detecting.
Ordered atoms replaced with prime numbers. Similar atoms has equal prime numbers.

Prime-codes of numbers now used in Morgan algorithm cycle.

On each loop for each atom calculated square of atom prime number multiplied to neighbors atoms prime numbers.
looping repeated while all atoms can't be unique or while number of unique atoms will not change for 3 times.

In [ ]:
ms2 = str(m2)  # get and print signature
print(ms2)  # or just print(reagent1)

hms2 = bytes(m2)  # get sha512 hashed signature bytes-string

String formatting supported.
Usable for reporting

In [ ]:
print(f'f string {m2}')  # use signature in string formatting
print('C-style %s' % m2)
print('fromat method {}'.format(m2))

In [ ]:
m2.reset_query_marks()
print(f'{m2:hn}')  # get signatures with hybridization and neighbors data
print('{:h}'.format(m2))  # get signature with hybridization only data
# h - hybridization marks, n- neighbors marks
format(m2, 'n')

s1 - atom has s hybridization with one neighbor  
d3 - atom has d hybridization with 3 neighbors  
see reset_query_marks above

Signatures of projections also supported

In [ ]:
f'{proj:h}'

#### Molecules comparable and hashable 
Comparison based on molecules signatures. Equal molecules has equal signatures.  
Strings in Python hashable. Therefore molecules also hashable.

NOTE: molecules can be changed. This can lead to unobvious behavior of the sets and dictionaries in which these molecules were placed before the change. Don't change molecules (standardize, aromatize, hydrogens and atoms/bonds changes) in sets and dicts or don't use this sets/dicts.

In [ ]:
m1 != m2

In [ ]:
m7 == m11

In [ ]:
m7 is m11  # this is not same objects!

In [ ]:
benzene == proj_copy

In [ ]:
len({m1, m2, m7, m11}) == 3 # create set of unique molecules

## 2.2. Reaction signatures
Signature is SMIRKS string in which molecules of reagents, reactants, products presented in canonical order

API same as for molecules

In [ ]:
str(r1)

## 2.3. CGR signature
Signatures is SMIRKS-like strings where atoms in reagents and products has same order

In [ ]:
str(cgr2)

C-C-[O-].C(=O)(-O)-C(=O)(-O).O-C-C  
C-C-O-C(=O)(.O)-C(=O)(.O)-O-C-C

You can see changed bonds

# 3. Structure standardization

## 3.1. Molecules

Molecules has `standardize` and `aromatize` methods.

Aromatize transform kekule representation of rings into aromatized

Standardize include aromatize and groups standardization. Next rules included:

    • Aromatic N-Oxide 	[#7;a:1]=[O:2]>>[#7+:1]-[#8-:2]
    • Azide 			[#7;A;X2-:1][N;X2+:2]#[N;X1:3]>>[#7:1]=[N+:2]=[#7-:3]
    • Diazo  			[#6;X3-:1][N;X2+:2]#[N;X1:3]>>[#6;A:1]=[N+:2]=[#7-:3]
    • Diazonium  		[#6]-[#7:1]=[#7+:2]>>[#6][N+:1]#[N:2]
    • Iminium  		[#6;X3+:1]-[#7;X3:2]>>[#6;A:1]=[#7+:2]
    • Isocyanate  		[#7+:1][#6;A-:2]=[O:3]>>[#7:1]=[C:2]=[O:3]
    • Nitrilium  		[#6;A;X2+:1]=[#7;X2:2]>>[C:1]#[N+:2]
    • Nitro  			[O:3]=[N:1]=[O:2]>>[#8-:2]-[#7+:1]=[O:3]
    • Nitrone Nitronate 	[#6;A]=[N:1]=[O:2]>>[#8-:2]-[#7+:1]=[#6;A]
    • Nitroso  		[#6]-[#7H2+:1]-[#8;X1-:2]>>[#6]-[#7:1]=[O:2]
    • Phosphonic  		[#6][P+:1]([#8;X2])([#8;X2])[#8-:2]>>[#6][P:1]([#8])([#8])=[O:2]
    • Phosphonium Ylide  	[#6][P-:1]([#6])([#6])[#6+:2]>>[#6][P:1]([#6])([#6])=[#6;A:2]
    • Selenite  		[#8;X2][Se+:1]([#8;X2])[#8-:2]>>[#8][Se:1]([#8])=[O:2]
    • Silicate  		[#8;X2]-[#14+:1](-[#8;X2])-[#8-:2]>>[#8]-[#14:1](-[#8])=[O:2]
    • Sulfine  		[#6]-[#6](-[#6])=[S+:1][#8-:2]>>[#6]-[#6](-[#6])=[S:1]=[O:2]
    • Sulfon  			[#6][S;X3+:1]([#6])[#8-:2]>>[#6][S:1]([#6])=[O:2]
    • Sulfonium Ylide  	[#6][S-:1]([#6])[#6+:2]>>[#6][S:1]([#6])=[#6;A:2]
    • Sulfoxide  		[#6][S+:1]([#6])([#8-:2])=O>>[#6][S:1]([#6])(=[O:2])=O
    • Sulfoxonium Ylide  	[#6][S+:1]([#6])([#8-:2])=[#6;A]>>[#6][S:1]([#6])(=[#6;A])=[O:2]
    • Tertiary N-Oxide  	[#6]-[#7;X4:1]=[O:2]>>[#6]-[#7+:1]-[#8-:2]


In [ ]:
m12

In [ ]:
m12.aromatize() # return number of transformed rings

In [ ]:
m12 # cache flushed automatically

In [ ]:
m12.standardize()  # return number of transformed groups

In [ ]:
m12

Molecules has `explicify_hydrogens` and `implicify_hydrogens` methods.

This methods add or remove hydrogens in molecule.

Currently implicit H atoms incorrectly calculated for pyrole-like molecules

In [ ]:
m1.explicify_hydrogens() # return number of added Hs

In [ ]:
m1 # added hydrogen atoms coordinates not calculated. Look to H atoms position on image

In [ ]:
m1.implicify_hydrogens() # return number of removed Hs

In [ ]:
m1

CGRtools has experimental algorithm for 2d geometry calcultaion. works fine only for small molecules.
require `numpy` and `scipy` packages

In [ ]:
m1.explicify_hydrogens()
m1.calculate2d() # experimental force-based 2d geometry calculation.
m1

## 3.2. Reactions standardization

In [ ]:
reactions[2]

In [ ]:
reactions[2].standardize()
reactions[2]

# 4. Isomorphism

## 4.1. Molecules Isomorphism
CGRtools has simple isomorphism API.

Equal atoms in isomorphism is atoms with same charge/multiplicity and isotope numbers

In [ ]:
m7

In [ ]:
m8

In [ ]:
benzene.standardize()
benzene

In [ ]:
# isomorphism operations
print(benzene < m7)  # benzene is substructure of m7
print(benzene > m7)  # benzene is not superstructure of m7
print(benzene <= m7) # benzene is substructure/or same structure of m7
print(benzene >= m7) # benzene is not superstructure/or same structure of m7
print(benzene < m8) # benzene is not substructure of m8. it's equal
print(benzene <= m8)

In [ ]:
m5

In [ ]:
m6

In [ ]:
m5.get_substructure_mapping(m6)  # mapping of m1 substructure into m2 superstructure

In [ ]:
for m in m5.get_substructure_mapping(m6, limit=0):  # iterate over all possible substructures
    print(m)
#limit is returned number of matches. 0 is all possible

In [ ]:
benzene.get_mapping(m8)  # mapping benzene into m8

## 4.2. Reactions
Don't support isomorphism. But molecules in reaction can be matched.

In [ ]:
try:
    m6 < r1
except TypeError:
    print(format_exc())

In [ ]:
any(m6 < m for m in r1.products) # check if any product side molecule is substructure of m6

In [ ]:
r1.products[0] # superstructure

In [ ]:
m6 # substructure

## 4.3 CGR
API same as for molecules

For CGR supported matching CGR in CGR and molecule in CGR.

Equal atoms in isomorphism is atoms with same charge/multiplicity and isotope numbers in reagent and product states

In [ ]:
m8 < cgr1

In [ ]:
cgr1 <= cgr1

## 4.4 Queries

In [ ]:
# for quering molecules need to calculate neighbors and hybridization
m9.reset_query_marks()
m10.reset_query_marks()

In [ ]:
m9 # acid

In [ ]:
m10 # ether

In [ ]:
carb

Molecules isomorphism don't take into account neighbors and hybridization

In [ ]:
carb < m9 # carb currently is molecule projection.

In [ ]:
carb < m10

You need to convert molecule (or it's projection) into QueryContainer object which support neighbors and hybridization data.

API of isomorphism same.

In [ ]:
q = QueryContainer(carb)  # convert molecule into query

In [ ]:
q < m9 # now neighbors and hybridization takes into account. 

Acid has Hydroxyl group which has one non-hydrogen neighbor. Our query require oxygen atom with two non-hydrogen neighbors.


In [ ]:
q < m10 # ether match

In [ ]:
m2.reset_query_marks()
m2

In [ ]:
q < m2

# 5. Transformation rules extraction

In [ ]:
cgr1.center_atoms # list of atom numbers of reaction center[s]

In [ ]:
cgr1.center_bonds # list of dynamic bonds as tuples of adjacent atom numbers

In [ ]:
cgr1.centers_list # list of list of atom numbers of each reaction centers

In [ ]:
rc1 = cgr1.substructure([13, 7]) # get reaction center. see molecules for substructure API
format(rc1, 'hn')

rc1 is phenol reduction

In [ ]:
rule = QueryCGRContainer(rc1)

In [ ]:
print(rule)

# 6. Reactor

Reactor objects store single transformation and can apply it to molecules or CGRs.

Transformations is ReactionContainer object which in reagent side consist query for matching group and in product side patch for updating matched atoms and bonds 

In [ ]:
from CGRtools import CGRreactor
from CGRtools.containers import *

## 6.1. Products generation
Reactor works similar to ChemAxon Reactions enumeration.

Lets try to etherificate acids.

First we need to construct carboxy group matcher query.  
Next ether group patch needed. 

Atom numbers in query and patch should be mapped. same atoms have same numbers.

In [ ]:
ether = QueryContainer() # this query match acids
ether.add_atom({'element': 'C', 'neighbors': 3})
ether.add_atom({'element': 'O', 'neighbors': 1})
ether.add_atom('O')
ether.add_bond(1, 2, 1) # This bond between carbon and hydroxyl oxygen
ether.add_bond(1, 3, 2)
print(ether)

In [ ]:
methyl_ether = MoleculeContainer()
methyl_ether.add_atom('C', 1) # second argument is predefined atom mapping
methyl_ether.add_atom('O', 2)
methyl_ether.add_atom('O', 4)
methyl_ether.add_atom('O', 3)
methyl_ether.add_atom('C', 5)
methyl_ether.add_bond(1, 4, 1)
methyl_ether.add_bond(1, 3, 2)
methyl_ether.add_bond(4, 5, 1)
# No bond between atom 1 and atom 2. Thi bond will be broke
methyl_ether.calculate2d()
methyl_ether

In [ ]:
m6.reset_query_marks() # need for correct matching
m6 # acid

In [ ]:
template = ReactionContainer([ether], [methyl_ether]) # template is Reaction
reactor = CGRreactor(template) # reactor initialized
reacted_acid = reactor(m6)

In [ ]:
reacted_acid.calculate2d(scale=2)
reacted_acid

You can see separate oxygen (water) and ether group.

For next group transformation call reactor with reacted molecule as argument

In [ ]:
reacted_acid.reset_query_marks() # this is new molecule and marks not set
second_stage = reactor(reacted_acid)
second_stage.calculate2d(scale=2)
second_stage

`second_stage` has 3 components in single object. We can split it into individual molecules and place all molecules into reaction object. 

In [ ]:
products = second_stage.split()
syntez = ReactionContainer([m6], products)
syntez.fix_positions()
syntez

For multicomponent reactions You can unite molecules of reagents into single molecule object and pass it to reactor.

For molecules with multiple matches possible to enumerate all combinations.

In [ ]:
m6copy = m6.copy()
m6copy.atom(5).isotope = 13 # isotope mark
m6copy.reset_query_marks()
enums = set()
for m in reactor(m6copy, limit=0): # limit=0 is enumeration of all matches
    print(m)
    m.calculate2d(scale=2)
    enums.update(m.split())
enums = list(enums)

In [ ]:
m6copy

In [ ]:
enums[0]

In [ ]:
enums[1]

In [ ]:
enums[2]

## 6.2. MetaReactions (reactions on CGRs).
Example of atom-to-atom mapping fix. 

In [ ]:
reactions[1]

In [ ]:
cgr = ~reactions[1]
print(cgr)

In [ ]:
cgr.centers_list # [10,11,12] - pseudo reaction

reaction has AAM error in nitro-group

![error.png](attachment:error.png)

Lets try to use Reactor for AAM fixing

In [ ]:
nitro = QueryCGRContainer() # construct query for invalid reaction center
nitro.add_atom({'element': 'N', 'charge': 1, 'p_charge': 1})
nitro.add_atom({'element': 'O', 'charge': 0, 'p_charge': -1}) # atom 2
nitro.add_atom({'element': 'O', 'charge': -1, 'p_charge': 0}) # atom 3
nitro.add_atom('C')

nitro.add_bond(1, 2, {'order': 2, 'p_order': 1})
nitro.add_bond(1, 3, {'order': 1, 'p_order': 2})
nitro.add_bond(1, 4, 1)
print(nitro)
# this query can match "transformation" in CGR shoved above


In [ ]:
nitro < cgr # query works!

In [ ]:
valid_nitro = MoleculeContainer() # construct fixer nitro group without dynamic atoms
valid_nitro.add_atom({'element': 'N', 'charge': 1})
valid_nitro.add_atom({'element': 'O', 'charge': -1})
valid_nitro.add_atom('O')

valid_nitro.add_bond(1, 2, 1)
valid_nitro.add_bond(1, 3, 2)
print(valid_nitro)

In [ ]:
valid_nitro.calculate2d()
valid_nitro # this is molecule. we will use it for updating matched nitro "transformation" into unchanged group

Now time to prepare **Template**.

Template is Reaction container with query in reagents and patch in products

In [ ]:
template = ReactionContainer([nitro], [valid_nitro])

CGRreactor class accept single template.


In [ ]:
reactor = CGRreactor(template)

CGRreactor object is callable and accept as argument molecule or CGR.

NOTE: `fixed` is new CGR object

In [ ]:
fixed = reactor(cgr) # fix CGR

CGRreactor returns None if query not matched else patched structure

In [ ]:
print(fixed)

`C-C(-I).O-C:1:C:C:C(:C:C:1)-[N+](-[O-])=O`  
`C-C(.I)-O-C:1:C:C:C(:C:C:1)-[N+](-[O-])=O`

You can see what nitro group now static

In [ ]:
fixed.centers_list # pseudo center disappeared

![cgr3.png](attachment:cgr3.png)

# 7. Input-output operations
*CGRtools.files* subpackage contains readers and writers classes.

## 7.1. MDL RDF reader

**RDFread** class can be used for RDF files reading.
Instance of this class is file-like object which support **iteration**, has a method **read()** for parsing all data and **context manager**.


### 7.1.1. Read file from disk

In [ ]:
from CGRtools.files import * # import all available readers and writers

with RDFread('example.rdf') as f:
    first = next(f)  # get first reaction. generator-like
    data = f.read()  # read remaining reactions to list

data = []
with RDFread('example.rdf') as f:
    for r in f:  # loop supported
        data.append(r)

with RDFread('example.rdf') as f:
    data = [r for r in f]  # list comprehensions. Note: use f.read() instead

#### OOP-stype Pathlib supported

In [ ]:
from pathlib import Path

with RDFread(Path('example.rdf')) as r:
    r = next(r)

#### opened files supported
For RDF file should be opened in text mode

In [ ]:
with open('example.rdf') as f, RDFread(f) as r:
    r = next(r)

### 7.1.2. Transparent loading from archives and network
Readers designed for transparently support of any type of data sources. 

Page https://cimm.kpfu.ru/seafile/f/aeaca685e3854ae2bbad/?dl=1 returns RDF file

Data sources should be file-like objects

In [ ]:
from requests import get
from io import StringIO

# get function return requested URL which has attribute text. 
# in example this text is whole RDF stored in single string.
# RDFread not support parsing of strings, but you can emulate files with data from strings by using io.StringIO
with StringIO(get('https://cimm.kpfu.ru/seafile/f/aeaca685e3854ae2bbad/?dl=1').text) as f, RDFread(f) as r:
    r = next(r)
    print(r, 'StringIO with downloaded from network data')

# python support gzipped data. this example shows how to work with compressed data without decompressing to disk.
from gzip import open as gzip_open
with gzip_open('example.rdf.gz', 'rt') as f, RDFread(f) as r:
    r = next(r)
    print(r, 'gzipped file')

# zip-files also supported out of the box 
# zipped files can be opened only in binary mode. io.TextIOWrapper can be used for transparent decoding into text
from zipfile import ZipFile
from io import TextIOWrapper
with ZipFile('example.zip') as z, z.open('example.rdf') as c:
    with TextIOWrapper(c) as f, RDFread(f) as r:
        r = next(r)
        print(r, 'zip archive')

# tar-file example
from tarfile import open as tar_open
from io import TextIOWrapper
with tar_open('example.tar.gz') as t:
    c = t.extractfile('example.rdf')
    with TextIOWrapper(c) as f, RDFread(f) as r:
        r = next(r)
        print(r, 'gzipped tar archive')

## 7.2. Other Readers
* SDFread - MOL, SDF files reader (v2000, v3000)
* MRVread - ChemAxon MRV files reader (lxml parser used)
* SMILESread - new-line separated SMILES strings files reader (coho backend used)
* INCHIread - new-line separated INCHI strings files reader (INCHI trust backend used)

All files except MRV should be opened in **text-mode**  
MRV requires binary mode `open('/path/to/data.mrv', 'rb')`

In [ ]:
with MRVread(open('example.mrv', 'rb')) as f:
    mrv = next(f)
mrv

## 7.3. File writers
Supported writing in next formats:
* RDFwrite (v2000) - molecules and reactions  write in RDF format
* SDFwrite (v2000) - molecules only  write in SDF format
* MRVwrite - molecules and reactions write in MRV format

Writers has same API as readers. all writers works with text-files
Writers has `write` method which accept as argument singe reactionor molecule or CGR object

In [ ]:
with RDFwrite('out.rdf') as f: # context manager supported
    for r in data:
        f.write(r)
# file will be overriden

In [ ]:
f = RDFwrite('out.rdf') # ongoing writing into single file
for r in data:
    f.write(r)

f.write(r1)

f.close() # close file. flush buffers.

## 7.4 CGR can be stored in MDL SDF and loaded from.

White-paper with SDF-CGR specification included.

In [ ]:
from CGRtools.files import *
from io import StringIO

with StringIO() as f,  SDFwrite(f) as w:
    w.write(cgr2)
    mdl = f.getvalue()
print(mdl)

In [ ]:
with StringIO(mdl) as f,  SDFread(f) as r:
    cgr3 = next(r)
print(cgr3)
print(type(cgr3))